In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
from collections import defaultdict
import datetime

import dateutil
import scipy
from dateutil.relativedelta import relativedelta

In [ ]:
alias_map = {}
clean = lambda x: ''.join(x.split()).lower()

In [ ]:
with open('arena_matches3.pkl','rb') as fp: #cs_matches
    results = pickle.load(fp)

In [ ]:

data = []
for k in results:
    basics = [k] + list(results[k][:3]) +[results[k][-1]]
    for m in results[k][3]:
        new_elem = basics + list(m)
        if new_elem[5] == '':
            new_elem[5] = new_elem[3]
        if new_elem[5] == '':
            continue
        t1 = clean(new_elem[6])
        t2 = clean(new_elem[7])
        
        
        new_elem[6] = alias_map.get(t1,t1)
        new_elem[7] = alias_map.get(t2,t2)
        

        data.append(new_elem)

In [ ]:
new_elem,len(data)

In [ ]:
df = pd.DataFrame(data,columns=['venue','tier','type','vtime','game','mtime','t1','t2','s1','s2','rnds'])
df['mtime'] = df['mtime'].map(lambda x: dateutil.parser.parse(x.replace('??','01')) if type(x) == str else x)
df['mtime'] = pd.to_datetime(df['mtime'],utc=True)

df['t1'] = df['t1'].map(lambda x: x.lower())
df['t2'] = df['t2'].map(lambda x: x.lower())


df = df[df['t1'] != 'bye']
df = df[df['t2'] != 'bye']
df = df[df['t1'] != '']
df = df[df['t2'] != '']
df = df[~df.t1.map(lambda x: 'bye' in x)]
df = df[~df.t2.map(lambda x: 'bye' in x)]
df = df[~df.t1.map(lambda x: 'tbd' == x)]
df = df[~df.t2.map(lambda x: 'tbd' == x)]
df = df[~df.t1.map(lambda x: 'bracketmatch' in x)]
df = df[~df.t2.map(lambda x: 'bracketmatch' in x)]

subs = [('dj:)','dj'),('destrukt','dkt'),('razerj','jibo'),('a1r-r41d','a1r_r41d'),('cooler','cooller')]
subs += [('toxic','toxjq'),('tox','toxjq'),('dragoax','dragaox'),('razerj','jibo'),('razerjibo','jibo'),('fox','fojji')]
subs += [('matr0xpl','matr0xpl'),('toxic(americanplayer){{!}}toxic','toxic_us'),('solll_','solll')]
subs += [('inz_', 'inz'), ('j_skey', 'jskey'), ('memento_mori', 'mementomori'), ('pie_hero', 'piehero'), ('septic_flesh', 'septicflesh'), ('sib_', 'sib'), ('st_germain', 'stgermain'), ('zanes_', 'zanes')]
subs += [('brute-', 'brute'),('dr.doom', 'drdoom'), ('ms.x', 'msx'),('1grayfoxz', 'grayfoxz'),('astz!', 'astz')]
subs += [('buggunsjoey', 'biggunsjoey')]
subs += [('bootss', 'boots'),('bootsss', 'boots'),('ch33rra', 'ch3rra')]
subs += [('zblazin', 'blazin'),('troolzy', 'troolz'),('vedmedik', 'vedemedik'),('vigur', 'vigurr'),('spawn2k', 'spawn2k4'),('spawn2k4', 'spawn2k'),('stenx', 'strenx'),('scoob', 'scooby'),('richochet', 'ricochet'),('sassmaster2', 'sassmaster'),('phosgenee', 'phosgene'),('overflux1', 'overflux'),('nonserium', 'nonservium'),('naaani', 'naani'),('maggotkil', 'maggotkill'),('k11lsen', 'k1llsen'),('k1lsen', 'k1llsen'),('lakermann', 'lakerman'),('d-pecka', 'pecka'),('dazatorr', 'dazator'),('defense', 'defense1'),('headbang3rr', 'headbang3r'),('infinte', 'infinite'),('karwik', 'karwick')]
subs += [('?joker', 'joker')]
subs += [('insan3', 'ins')]
subs += [('ballin4life_', 'ballin4life')]
subs += [('janky','jankygoatsmugla')]
subs += [('jankygoat','jankygoatsmugla')]
subs += [('koda','cody')]
subs += [('p0ngmeister','p0ng')]
subs += [('antreax2','strongsage')]
subs += [('vigi1ante','vig1lante')]
subs += [('quake2rizzy','rizzy')]

for old, new in subs:
    df.loc[df['t1'] == old,'t1'] = new
    df.loc[df['t2'] == old,'t2'] = new

df.loc[(df.t1 == 'ins') & df.mtime.map(lambda x: x.year < 2007),'t2'] = 'insane'
df.loc[(df.t2 == 'ins') & df.mtime.map(lambda x: x.year < 2007),'t2'] = 'insane'
    
df.loc[df['game'] == 'db','game'] = 'dbt'
df.loc[df['game'] == 're','game'] = 'reflex'
df.loc[df['game'] == 'dmb','game'] = 'doombringer'

df.loc[df['game'].map(lambda x: x in['ut99','ut2k3','ut2k4']),'game'] = 'unreal'


if False:
    df = df[df.game != 'dbt']
    df = df[df.game != 'reflex']
    df = df[df.game != 'avp2']
    df = df[df.game != 'wsw']
    df = df[df.game != 'ws']
    df = df[df.game != 'ut3']
    df = df[df.game != 'doom2']
    df = df[df.game != 'doombringer']
    df = df[df.game != 'd2']

      

df.loc[df['tier'] != '1','tier'] = 'O'

df.loc[df.venue.map(lambda x: 'XSReality' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'MediaMarkt' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'Invitational' in x),'tier'] = '1'
df.loc[df.venue.map(lambda x: 'LAN Arena' in x),'tier'] = 'O'
df.loc[df.venue.map(lambda x: 'Atomic' in x),'tier'] = 'O'

#df = df[df.tier.isin(['A','S'])]
df['type'] = df['type'].map(lambda x: 'Online' if 'Offline' != x else 'Offline')
df = df.drop(df[(df.game == 'dbt') & (df.mtime.map(lambda x: x.year < 2021)) & (df.mtime.map(lambda x: x.month < 9))].index)
print(df.shape)


In [ ]:
df.tier

In [ ]:
df[df.venue == 'QuakeCon/2015/Quake Live/Duel/Grand Master']

In [ ]:
df.loc[df.venue == 'Quake Pro League/Season 1/Stage 3/Finals','type'] = 'Offline'
df.loc[df.venue == 'Quake Pro League/Season 1/Stage 4/Finals','type'] = 'Offline'
df.loc[df.venue == 'Quake World Championship/2020','type'] = 'Offline'
df.loc[df.venue == 'Quake Pro League/Season 2/Stage 1/Finals','type'] = 'Offline'
df.loc[df.venue == 'Quake Pro League/Season 2/Stage 2/Finals','type'] = 'Offline'


for tts in [_ for _ in df[df.game == 'dbt'].venue.unique() if 'Test Tournament Series' in _]:
    df.loc[df.venue == tts,'type'] = 'Offline'


In [ ]:
from datetime import date
import matplotlib.pyplot as plt
tweight2= np.arange(df.shape[0])
tweight2=tweight2/tweight2.max()/5 + 1
tweight2=tweight2/tweight2.max()
tweight2

tweight = (pd.Timestamp(date.today(),tz='UTC')-df.mtime).map(lambda x: x.days)
plt.hist(tweight,30)
#tweight = (np.tanh(-(tweight-120)/360) +1)/2
tweight = np.exp(-tweight/1000)*tweight2

tweight = tweight/tweight.max()
gweight = ((df.game == 'qc')  ).astype(int) + 1e-3
eweight = ((df['type'] == 'Offline')  ).astype(int)*2 + 1

names = sorted(list(set(list(df.t1.unique()) + list(df.t2.unique()))))
lenN = len(names)
n_idx = {k:v for v,k in enumerate(names)}

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df.mtime,tweight*eweight*gweight)
plt.xlim(left=datetime.date(2020,1,1))


In [ ]:
from scipy.sparse import dok_matrix

#S = dok_matrix((len(df), lenN), dtype=np.float32)
S = np.zeros((len(df), lenN), dtype=np.float32)
labels = []
for i,row in enumerate(df.itertuples()):
    t1 = n_idx[row[7]]
    t2 = n_idx[row[8]]
    s1 = row[9]
    s2 = row[10]
    S[i,t1] = 1
    S[i,t2] = -1
    #labels.append(int(s1 > s2))
    labels.append(max(0,s1)/(max(0,s1)+max(0,s2)))
labels = np.array(labels)[:,None]
S.shape,labels.shape

In [ ]:
#gweight = np.ones_like(gweight)
#tweight = np.ones_like(tweight)

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

X = torch.Tensor(S)
y = torch.Tensor(labels)
w = torch.Tensor((gweight*tweight*eweight)[:,None])


criterion = nn.BCELoss(reduction='none')  
model = nn.Linear(S.shape[1], 1)
optimizer = torch.optim.LBFGS([model.weight])
m = nn.Sigmoid()

num_epochs =100
for epoch in range(num_epochs):
        
    # Forward + Backward + Optimize
    def closure():
        optimizer.zero_grad()
        outputs = m(model(X))
        loss = criterion(outputs, y)
        loss_class_weighted = loss * w
        loss_class_weighted = loss_class_weighted.mean() + (model.weight**2).mean()/50
        loss_class_weighted.backward()
        print(loss_class_weighted)
        return loss_class_weighted

    optimizer.step(closure)
        
    #print(loss_class_weighted)
outputs = m(model(X))

In [ ]:
coef = model.weight.detach().numpy().ravel() # clf.coef_.ravel()

coef.shape

In [ ]:
#tourney = ['drrifted','skabuda','vig1lante','zorakwar','tiltedthanks','erebux','linchpin','ardi','vonslakken','rizzy','blindlight','xzist','lampyre','ballin4life','kk_horo','rey','thump4','flame','fortiss','minikieff','jagriff','coffinfiller','p0ng','davjs','julbars','kev','jojtherat','angrytetris','en1gma','jankygoatsmugla','ohurcool','jagriff','vexatiousluck','ryunyaan']
#tourney = ['rey','julbars','ballin4life','jagriff','kev','ohurcool','davjs','jankygoatsmugla']
tourney = ['whaz','malinkifox','supremegti','yup','davjs','georgi3','boots','v0dkax','strongsage','chance','cirux','deathr0w','festiveturtle','charle20','veske','korssa1r']
#tourney = ['rmv27','st0n3','pasita','inz','toxjq','arseny','enesy','phaze','xron','evmadic','pepco','klyb','prox1mo','ins','overtime','bukster']
tset = set(tourney)
sort_lr = sorted([(round(z*173+1500),n) for n,z in zip(names,coef)],reverse=True)
[_ for _ in sort_lr if _[1] in tset]


In [ ]:
import scipy.stats
predProbs = outputs.detach().numpy()#clf.predict_proba(S)
weightS = gweight*tweight
weightS = weightS/weightS.max()
V = np.diagflat(np.product(predProbs, axis=1))

sW = scipy.sparse.diags(weightS)

wS =  sW @ S
cov = wS.T @ V @ wS 
covLogit = np.linalg.inv(cov+np.identity(cov.shape[0])/5)

sigma_estimates = np.sqrt(np.diag(covLogit))

z_scores = coef/sigma_estimates # z-score for eaach model coefficient
p_values = [scipy.stats.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sort_lr = sorted([(round(z*173+1500),n,s*173) for n,z,s in zip(names,coef,sigma_estimates)],reverse=True)

resT = [_ for _ in sort_lr if _[1] in tset] + [(-1,n,0) for n in tourney if n not in names]
print(len(resT))
for a,b,c in resT:
    print('{:4d} ±{:3d}\t{}'.format(a,round(c),b))
#[_ for _ in tset if _ not in names]

In [ ]:
resT2 = sorted([_ for _ in sort_lr if _[1] in names],reverse=True)
for a,b,c in resT2[:50]:
    print('{:4d} ±{:3d}\t{}'.format(a,round(c),b))

In [ ]:
df[(df.t1 == 'jojtherat') | (df.t2 == 'jojtherat')]

In [ ]:
[_ for _ in names if 'tilt' in _]

In [ ]:
from collections import Counter

pc = Counter(list(df[(df.game == 'dbt')].t1) + list(df[(df.game == 'dbt')].t2))
sorted([(v,k) for k,v in pc.items()],reverse=True)


In [ ]:
[_ for _ in pc if 'vig' in _]

In [ ]:
df.game.value_counts()

In [ ]:
num_events = dict(df.tier.value_counts())
num_type = dict(df.type.value_counts())
N = df.shape[0]
num_type

score_mul = {  }
for t in num_type:
    for e in num_events:
        score_mul[(e,t)] = (N/num_type[t]) * (N/num_events[e])
        if t == 'Offline':
            score_mul[(e,t)]  *= 5
        if e == '1':
            score_mul[(e,t)]  *= 2
score_mul


In [ ]:
df = df.sort_values('mtime',0,kind='mergesort')

In [ ]:
df[((df.s1 < 0 ) | (df.s2  < 0))]

In [ ]:
import numpy as np
import datetime
# opt'd, low online
initial_v2= np.array([4.7467349168917297e+00, 2.8868572247883139e+02,
       6.4308658654774092e+01, 3.5468460498078309e+02,
       1.0815731752527216e+02, 1.5370825990888963e+02,
       1.1454452976515610e+01, 9.6774122757823349e+01,
       2.4091686971980901e+00, 9.7201728546434940e-01,
       7.6030441601557150e+00, 1.1510774881638908e+00,
       2.3782641811627565e+00, 2.9961924748222518e+01,
       1.1364910564528891e+01, 1.9505842310367453e-02,
       5.1463218032083313e-01, 3.4836163497475772e-02,
       9.7760304203004145e-01])
initial_v2 = np.array([5.11376841e+00, 3.34361568e+02, 6.81220696e+01, 3.44940340e+02,
       1.00599206e+02, 1.10393118e+02, 1.05370386e+01, 8.91482963e+01,
       1.36000055e+00, 1.06277688e+00, 8.44375624e+00, 1.24040835e+00,
       2.26389484e+00, 2.41575562e+01, 9.13975773e+00, 2.27318022e-02,
       5.57500119e-01, 3.58215040e-01, 9.24151215e-01])
sigmoid = lambda x: 1/(1+10**(-x/400)) 

def sigmoid(x):
    "Numerically-stable sigmoid function."
    if x >= 0:
        z = 10**(-x/400.0)
        return 1 / (1 + z)
    else:
        z = 10**(x/400.0)
        return z / (1 + z)
def benchmark_func(x):
    #x = x*initial_v2
    ratings = defaultdict(lambda: 1500)
    ratings_game = defaultdict(lambda: 1500)

    prev_update = {}
    x = (np.array(x).flatten())
    

    #print(blend,amount)

    x = abs(x)
    per_ratings = {
        ('O','Online'): x[0],
        ('O','Offline'): x[1],
        ('1','Online'): x[2],
        ('1','Offline'): x[3],
    }
    per_ratings_const = {
        ('O','Online'): x[4],
        ('O','Offline'): x[5],
        ('1','Online'): x[6],
        ('1','Offline'): x[7],
    }
    power = x[8]
    power2 = x[9]
    cMean = 100*x[10]
    cVar =  x[11]
    blend= np.exp(-x[12])
    amount= np.exp(-x[13])
    
    fix_hack = np.exp(-x[14])
    fix_hack2 = np.exp(-x[15])
    blend2 = np.exp(-x[16])
    blend3 = np.exp(-x[17])
    target_reset = np.exp(-x[18])
    #print(target_reset)
    score = 0.0

    correct = 0
    total_g = 0

    for row in df.itertuples():
        game = row[5]
        t1 = row[7] #+ '_' + row[5]
        t2 = row[8] #+ '_' + row[5]
        cTime = row[6].replace(tzinfo=None)

        total = row[9]+row[10]
        bot,top = sorted([row[9],row[10]])
        diff = top - bot
        if total < 1 or diff <1:
            prev_update[t1] = cTime
            prev_update[t2] = cTime
            continue
        diff /= total

        # per game ratings
        t1g = t1 + '_' + game
        t2g = t2 + '_' + game
        
        if t1g in ratings_game:
            r1 = ratings_game[t1g]
            time1 = prev_update.get(t1g,cTime)
        else:
            r1 = ratings[t1]*blend2+1500*(1-blend2)
            time1 = prev_update.get(t1,cTime)

            
        if t2g in ratings_game:
            r2 = ratings_game[t2g]
            time2 = prev_update.get(t2g,cTime)
        else:
            r2 = ratings[t2]*blend2+1500*(1-blend2)  
            time2 = prev_update.get(t2,cTime)
        
        # time adjust
        td1 = float((cTime - time1).days)
        to1500_1 = float(sigmoid((td1-cMean)*cVar))*target_reset
        r1 = (to1500_1)*1500 + (1-to1500_1)*r1

        td2 = float((cTime - time2).days)
        to1500_2 = float(sigmoid((td2-cMean)*cVar))*target_reset
        r2 = (to1500_2)*1500 + (1-to1500_2)*r2



        rdiff = abs(r1-r2)
        p1 = sigmoid(r1-r2)
        p2 = 1 - p1
        K = (diff**power) * (per_ratings[row[2:4]] ) + per_ratings_const[row[2:4]]
        K = K*blend + amount*rdiff*(1-blend)

        if top < 2 and top > 1:
            K *= fix_hack
        if top < 3 and top > 2:
            K *= fix_hack2
        #print(blend)
        #raise
        #if r1+r2 > 3400:

        #else:
        #    correct += 0.5
        #    total_g += 1
        sMul = score_mul[row[2:4]] * max(0.1,(r1+r2)/3000.0)**2
        #print(K,sMul)

        correct += int((row[9] > row[10]) == ( r1 > r2)) *sMul
        total_g += sMul

        if row[9] > row[10]:
            E = p2
            ratings[t1] = r1 + K * E**power2
            ratings[t2] = r2 - K * E**power2

            score += -np.log(max(p1,5e-2)) * sMul
        else:
            E = p1
            ratings[t1] = r1 - K * E**power2
            ratings[t2] = r2 + K * E**power2

            score += -np.log(max(p2,5e-2)) * sMul
        prev_update[t1] = cTime
        prev_update[t2] = cTime
        prev_update[t1g] = cTime
        prev_update[t2g] = cTime
        ratings_game[t1g] = ratings[t1]
        ratings_game[t2g] = ratings[t2]
    return score/max(1e-3,(correct)/total_g-0.5)**0.25#,ratings
initial_v = np.ones(19).astype(np.float32)
initial_v =initial_v2

s= benchmark_func(initial_v)
s,s<385566.4

In [ ]:
#import cma #
#es = cma.CMAEvolutionStrategy(initial_v, 0.01, {'tolx':1e-6,'tolfun':1e-6,})
#es.optimize(benchmark_func,n_jobs=0)

In [ ]:
# you might wonder why two versions of the same code
# this is a silly way for me to force myself to do code review for myself
from collections import defaultdict
ratings = defaultdict(lambda: 1500)
rt = defaultdict(list)
ratings_game = defaultdict(lambda: 1500)
prev_update = {}
top_games = []

per_ratings = {
    ('O','Online'): initial_v[0],
    ('O','Offline'): initial_v[1],
    ('1','Online'): initial_v[2],
    ('1','Offline'): initial_v[3],
}
per_ratings_const = {
    ('O','Online'): initial_v[4],
    ('O','Offline'): initial_v[5],
    ('1','Online'): initial_v[6],
    ('1','Offline'): initial_v[7],
}
power = initial_v[8]
power2 = initial_v[9]
cMean = 100*initial_v[10]
cVar =  initial_v[11]
blend= np.exp(-initial_v[12])
amount= np.exp(-initial_v[13])

fix_hack = np.exp(-initial_v[14])
fix_hack2 = np.exp(-initial_v[15])
blend2 = np.exp(-initial_v[16])
blend3 = np.exp(-initial_v[17])
target_reset = np.exp(-initial_v[18])

print(fix_hack)
score = 0.0

correct = 0
total_g = 0

for row in df.itertuples():
    game = row[5]
    t1 = row[7] #+ '_' + row[5]
    t2 = row[8] #+ '_' + row[5]
    cTime = row[6].replace(tzinfo=None)
    time1 = prev_update.get(t1,cTime)
    time2 = prev_update.get(t2,cTime)

    total = row[9]+row[10]
    diff = abs(row[9]-row[10])
    if total < 1 or diff <1:
        prev_update[t1] = cTime
        prev_update[t2] = cTime
        continue
    diff /= total

    
    # per game ratings
    t1g = t1 + '_' + game
    t2g = t2 + '_' + game

    if t1g in ratings_game:
        r1 = ratings_game[t1g]
        time1 = prev_update.get(t1g,cTime)
    else:
        r1 = ratings[t1]*blend2+1500*(1-blend2)
        time1 = prev_update.get(t1,cTime)


    if t2g in ratings_game:
        r2 = ratings_game[t2g]
        time2 = prev_update.get(t2g,cTime)
    else:
        r2 = ratings[t2]*blend2+1500*(1-blend2)  
        time2 = prev_update.get(t2,cTime)

    # time adjust
    td1 = float((cTime - time1).days)
    to1500_1 = float(sigmoid((td1-cMean)*cVar))*target_reset
    r1 = (to1500_1)*1500 + (1-to1500_1)*r1

    td2 = float((cTime - time2).days)
    to1500_2 = float(sigmoid((td2-cMean)*cVar))*target_reset
    r2 = (to1500_2)*1500 + (1-to1500_2)*r2

    
    top_games.append( (r1/2+r2/2,t1,t2,game,cTime) )
    
    if td1 > 300:
        rt[t1].append((cTime,np.nan,row[5]))
    if td2 > 300:
        rt[t2].append((cTime,np.nan,row[5]))
    
    rdiff = abs(r1-r2)
    p1 = sigmoid(r1-r2)
    p2 = 1 - p1
    K = (diff**power) * (per_ratings[row[2:4]] ) + per_ratings_const[row[2:4]]
    K = K*blend + amount*rdiff*(1-blend)

    bot,top = sorted([row[9],row[10]])
    if top < 2 and top > 1:
        K *= fix_hack
    if top < 3 and top > 2:
        K *= fix_hack2
    #print(blend)
    #raise
    #if r1+r2 > 3400:

    #else:
    #    correct += 0.5
    #    total_g += 1
    sMul = score_mul[row[2:4]] * max(0.1,(r1+r2)/3000.0)**2
    #print(K,sMul)

    correct += int((row[9] > row[10]) == ( r1 > r2)) *sMul
    total_g += sMul

    if row[9] > row[10]:
        E = p2
        ratings[t1] = r1 + K * E**power2
        ratings[t2] = r2 - K * E**power2

        score += -np.log(max(p1,5e-2)) * sMul
    else:
        E = p1
        ratings[t1] = r1 - K * E**power2
        ratings[t2] = r2 + K * E**power2

        score += -np.log(max(p2,5e-2)) * sMul
    prev_update[t1] = cTime
    prev_update[t2] = cTime
    prev_update[t1g] = cTime
    prev_update[t2g] = cTime
    ratings_game[t1g] = ratings[t1]
    ratings_game[t2g] = ratings[t2]

    rt[t1].append((cTime,ratings[t1],row[5]))
    rt[t2].append((cTime,ratings[t2],row[5]))
score,100*correct/total_g,score*(1-correct/total_g),score/max(1e-3,(correct)/total_g-0.5)**0.25#,ratings

# 57841.09221530504
# per_ratings = {('A','Online'): 6.35,
#               ('S','Online'): 29.43,
#               ('A','Offline'): 8.56,
#               ('S','Offline'): 17.6}

#const = 14.86
#weeks = 45
#power = 2.24
# 17185 w/ 61.8 to get 6550 (no games)
# 16616 w/ 59.1 to get 6801 (games)
# 15940 w/ 56.14 to get 6991 (no games)
# 15986 w/ 53.72 to get 7397.8 (games)

# 16991 w/ 64.9 to get 5966 and 11414 (games)
# 17172 w/ 68.1 to get 5470 and 94649 (no games)357033

In [ ]:
sorted(top_games,reverse=True)

In [ ]:
dbt = []
for k,v in ratings_game.items():
    if k[-4:] == '_dbt':
        dbt.append((round(v),k))
sorted(dbt,reverse=True)

In [ ]:
sorted([_ for _ in dbt if _[1][:-4] in tset],reverse=True)

In [ ]:
import matplotlib.pyplot as plt
#cMean,cVar = 1000,1
xn = np.linspace(0,365*5)
yn = [sigmoid((_-cMean)*cVar)*target_reset for _ in xn]
plt.plot(xn,yn)
cMean,cVar,initial_v[10],initial_v[11]


In [ ]:
sorted([(v,k) for k,v in ratings.items()],reverse=True)

In [ ]:
top_teams = []
for p,res in rt.items():
    for game in df.game.unique():
        if game == 'q3':
            v = [_ for _ in res if _[2] == game  and _[0].year < 2006 and np.isfinite(_[1])]
        else:
            v = [_ for _ in res if _[2] == game and np.isfinite(_[1])]

        if p =='fojji' and game == 'q3':
            pass#raise
        if len(v) > 0:
            #v = v[len(v)//3:]
            best_idx = np.argmax(np.array(v)[:,1])
            top_teams.append((v[best_idx][1:],p,v[best_idx][0]))
top_teams = sorted(top_teams,reverse=True)
#top_teams = sorted([(v[np.argmax(np.array(v)[:,1])][1:],k) for k,v in rt.items() if len(v) > 0],reverse=True)
top_teams

In [ ]:
df[((df.t1 == 'julbars') & (df.t2 == 'rey')) | ((df.t1 == 'rey') & (df.t2 == 'julbars')) ]

In [ ]:
df[((df.t1 == 'strenx') | (df.t2 == 'strenx')) &  (df.game == 'ql') &  (df.mtime.map(lambda x: x.year == 2011))]

In [ ]:
#df[df.venue == 'Quake Pro League/Season 1/Stage 1/Finals']
rt['julbars']

In [ ]:
color_dict = defaultdict(lambda: ('#ff00ff'))
color_dict['zero4'] = '#cc3311'
color_dict['fatal1ty'] = '#66CCEE'
color_dict['cooller'] = '#CCBB44'
color_dict['cypher'] = '#AA3377'
color_dict['rapha'] = '#4477AA'
color_dict['toxjq'] = '#228833'
color_dict['vo0'] = '#EE8866' 

color_dict['czm'] = '#6ab04c'
color_dict['z4muz'] = '#e58e26'
color_dict['lexer'] = '#6555e0'
color_dict['av3k'] = '#0077bb'
color_dict['dahang'] = '#663333'
color_dict['fojji'] = '#EE6677'
color_dict['ztrider'] = '#222255'
color_dict['gellehsak'] = '#bb5566'
color_dict['stermy'] = '#bbcc33'
color_dict['zyz'] = '#ddaa33'


#'#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE','#AA3377',

In [ ]:
import matplotlib.pyplot as plt
import datetime
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from collections import OrderedDict
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
for game in df.game.unique():#['q3','pk','q4','ql','qc']:
    plt.figure(figsize=(10,5),dpi=120)
    tt = [_ for _ in top_teams if game == _[0][1]]
    if game == 'qc':
        tt = [_ for _ in tt if _[1] != 'evil']
    for i in range(min(len(tt),6)):
        tm = tt[i][1]
        team_plot = np.array([_ for _ in rt[tm] if _[2] == game and np.isfinite(_[1])])
        c = color_dict[tm]
        plt.plot(team_plot[:,0],team_plot[:,1],label=tm,lw=5.5,alpha=0.72)

        team_plot = team_plot[min(5,len(team_plot)-1):]
        argmin = np.argmax(team_plot[:,1])
        peak = team_plot[argmin]
        
        plt.scatter([peak[0]],[peak[1]],c=plt.gca().lines[-1].get_color(),s=200)
    if game == 'qc' and False:
        tm = 'clawz'
        team_plot = np.array([_ for _ in rt[tm] if _[2] == game and np.isfinite(_[1])])
        c = color_dict.get(tm,'#f15bb5')
        plt.plot(team_plot[:,0],team_plot[:,1],label=tm,lw=5.5,alpha=0.72,c=c)

        team_plot = team_plot[min(5,len(team_plot)-1):]
        argmin = np.argmax(team_plot[:,1])
        peak = team_plot[argmin]

        plt.scatter([peak[0]],[peak[1]],c=plt.gca().lines[-1].get_color(),s=200)
    plt.legend(ncol=6,loc='lower center')
    plt.ylabel('Elo',weight='semibold')
    locator = mdates.YearLocator()
    # Specify the format - %b gives us Jan, Feb...
    fmt = mdates.DateFormatter('%Y')


    X = plt.gca().xaxis
    X.set_major_locator(locator)
    X.set_major_formatter(fmt)

    if game in ['pk','q4']:
        #plt.xlim(datetime.datetime(2018,9,1),datetime.datetime(2019,1,1))
        locator = mdates.MonthLocator(interval=4)
        # Specify the format - %b gives us Jan, Feb...
        fmt = mdates.DateFormatter('%b%Y')


        X = plt.gca().xaxis
        X.set_major_locator(locator)
        X.set_major_formatter(fmt)
    #if game == 'ql':
    plt.ylim(bottom=1500)
    games_dict = {'unreal':'Unreal Tournament','pk':'Painkiller','ql':"Quake Live",'q2':'Quake 2','q3':'Quake 3','q4': 'Quake 4','qc':'Quake Champions'}
    plt.title('Top 1v1 players in ' +games_dict.get(game,game),weight='medium')
    if game == 'ql':
        plt.xlim(datetime.datetime(2009,6,1), datetime.datetime(2016,10,1))
    plt.tight_layout()
    plt.savefig(game+'.png')

In [ ]:
tt

In [ ]:
rt['zorakwar']

In [ ]:
df[(df.t1=='cypher') & (df.mtime.map(lambda x: x.year ==2018)) & (df.mtime.map(lambda x: x.month ==8))]

In [ ]:
sorted([(([(0,0)] +[(_[1],_[0]) for _ in v if _[-1] == 'qc'] )[-1],k) for k,v in rt.items()],reverse=True)

In [ ]:
rt['cooller']

In [ ]:
[_ for _ in top_teams if _[0][1] == 'q3']

In [ ]:
df[df.mtime.map(lambda x: x.year == 2017) & (df.game == 'qc') & ((df.t1 == 'cooller') | (df.t2 == 'cooller'))  ]

In [ ]:
df[(((df.t1 == 'cypher') & (df.t2 == 'rapha')) | ((df.t1 == 'rapha') & (df.t2 == 'cypher'))) & (df.game == 'ql') ]

In [ ]:
sorted([(v,k) for k,v in ratings.items()],reverse=True)

In [ ]:
def get_top_at(time,team_data,timedel = datetime.timedelta(weeks=30)):
    td = timedel
    rt = team_data
    time = tm
    team_idx = {k: abs(np.array([_[0].replace(tzinfo=None) for _ in v])-tm) for k,v in rt.items() if len(v) > 0}
    team_idx = {k:np.argmin(v) for k,v in team_idx.items() if sum(v < td) > 1}
    top_teams_year = sorted([(round(np.nan_to_num(rt[k][i][1])),k) for k,i in team_idx.items()],reverse=True)
    return top_teams_year

In [ ]:
tm = datetime.datetime(month=5,year=2021,day=1)
top_teams_year = get_top_at(tm,rt)
print(tm,top_teams_year[:13])

In [ ]:
for yr in range(2020,2020):
    tm = datetime.datetime(month=1,year=yr+1,day=1)
    top_teams_year = get_top_at(tm,rt)
    print(tm,top_teams_year[:5])

In [ ]:
_ = plt.hist(df.mtime,100)

In [ ]:
import math

max_year = defaultdict(dict)
for k in rt:
    yrs = set([int(_[0].year) for _ in rt[k]])
    yrs = list(sorted(yrs))
    tm_d = {}
    for y in yrs:
        max_year[y][k] = max([(_[1],_[0]) for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ])

const_W = 1
alt_year = defaultdict(dict)
for k in rt:
    yrs = set([int(_[0].year) for _ in rt[k]])
    yrs = list(sorted(yrs))
    tm_d = {}
    for y in yrs:
        valid_g = [_[1] for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ]
        weights = [const_W] + [1 if valid_g[i] > valid_g[i-1] else const_W for i in range(1,len(valid_g))]
        weights = np.array(weights)
        weights = weights/(np.mean(weights))
        valid_g = np.array(valid_g) * np.array(weights)
        idx = np.argsort(valid_g)#[1:-1]
        #if k == 'czm' and y == 2008:
        #   raise
        valid_g = valid_g[idx]
        weights = weights[idx]
        #num_games  = len([_ for _ in rt[k] if _[0].year == y])**0.15
        if len(valid_g) > 5:
            #alt_year[y][k] =  np.mean(valid_g)
            alt_year[y][k] = ( np.mean([_**2 for _ in valid_g]))**(1/2)
        #alt_year[y][k] = np.sqrt(np.mean([_[1]**2 for _ in rt[k] if _[0].year == y and not math.isnan(_[1]) ]))


In [ ]:
rt['unkind']

In [ ]:
#df[((df.t1=='rix') | (df.t2 == 'rix')) & (df.mtime.map(lambda x: x.year < 2002))]
#rt['makaveli']

In [ ]:
for yr in range(1995,2022):
    top_teams_year = sorted([(v,k) for k,v in alt_year[yr].items()  if 'sweden' not in k],reverse=True)
    if len(top_teams_year) < 1:
        continue
    print('* **' +str(yr) + '**',end='\t')#,top_teams_year[:5])
    i=0
    sidx = min(5,len(top_teams_year))
    for s,n in top_teams_year[:5]:
        i+=1
        entk = ', ' if i!=sidx else '\n'
        print('{} ({})'.format(n,int(s)),end=entk)

In [ ]:
max_best_ever = []
for yr, res in max_year.items():
    for tm, pelo in res.items():
        #if yr != 2000:
        #    continue
        max_best_ever.append( (pelo[0],pelo[1].strftime("%Y-%m-%d"),str(yr) +'__' + tm))
sorted(max_best_ever,reverse=True)[:150]

In [ ]:
max_best_ever = []
for yr, res in max_year.items():
    for tm, pelo in res.items():
        max_best_ever.append( (pelo[0],pelo[1].strftime("%Y-%m-%d"),str(yr) +'__' + tm))
sorted(max_best_ever,reverse=True)[:50]

In [ ]:
df[df.venue.map(lambda x: 'intel' in x.lower()) & (df.type=='Online')].venue.unique()

In [ ]:
alt_best_ever = []
for yr, res in alt_year.items():
    for tm, pelo in res.items():
        alt_best_ever.append( (pelo,str(yr) +'__' + tm))
sorted(alt_best_ever,reverse=True)[:50]

In [ ]:
df[df.venue == 'Quake Pro League/Season 1/Stage 2/Finals']

In [ ]:
top_teams

In [ ]:
def get_top_year_elo(tm,yr,rt):
    return max([(_[1],_[0]) for _ in rt[tm] if _[0].year == yr])

In [ ]:
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
from collections import OrderedDict
import matplotlib.pyplot as plt

In [ ]:
color_dict['av3k'] ='#f15bb5'
color_dict['socrates'] = '#293241'
color_dict['mojo'] ='#f15bb5'


In [ ]:
df[(df.t1 == 'koda') | (df.t2 == 'koda')]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
names = ['rey','warden','zorakwar','davjs','ballin4life','ohurcool','julbars']
cmap = plt.get_cmap("tab10")
cmaps = [cmap((i)/float(len(names))) for i in range(len(names))]
years = [range(2015,2022) for i in range(len(names))]
for n,y,c in zip(names,years,cmaps):
    print(list(y))
    vs = np.array([_ for _ in rt[n] if _[0].year in list(y)])
    #c = color_dict[n]
    plt.plot(vs[:,0],vs[:,1],alpha=0.95,label=n,c=c,lw=5)
    for yr in y:
        top_teams_year = sorted([(v,k) for k,v in alt_year[yr].items()],reverse=True)
        for ratingT, tm in top_teams_year[:0]:
            if tm in names:
                idx = names.index(tm)
                if yr in years[idx]:
                    continue
            ratings = np.array([_ for _ in rt[tm] if _[0].year == yr])
            plt.plot(ratings[:,0],ratings[:,1],c='k',alpha=0.08,lw=1.5)
plt.legend(loc=4,ncol=7)
plt.ylim(bottom=1200)
plt.xlim(left=datetime.datetime(2020,1,1))
plt.ylabel('Elo')
plt.title('across all games (including doombringer lol)')
#plt.title('')


In [ ]:
plt.figure(figsize=(12,4))
names = ['fatal1ty','zero4','cooller','vo0','toxjq','cypher','rapha']
cmap = plt.get_cmap("tab10")
cmaps = [cmap((i)/float(len(names))) for i in range(len(names))]
years = [range(1999,2008),range(2000,2005),range(2003,2021),range(2004,2006),range(2006,2010),range(2008,2021),range(2009,2021),range(2000,2020)]
for n,y,c in zip(names,years,cmaps):
    print(list(y))
    vs = np.array([_ for _ in rt[n] if _[0].year in list(y)])
    c = color_dict[n]
    plt.plot(vs[:,0],vs[:,1],alpha=0.95,label=n,c=c,lw=5)
    for yr in y:
        top_teams_year = sorted([(v,k) for k,v in alt_year[yr].items()],reverse=True)
        for ratingT, tm in top_teams_year[:10]:
            if tm in names:
                idx = names.index(tm)
                if yr in years[idx]:
                    continue
            ratings = np.array([_ for _ in rt[tm] if _[0].year == yr])
            plt.plot(ratings[:,0],ratings[:,1],c='k',alpha=0.08,lw=1.5)
plt.legend(loc=4,ncol=7)
plt.ylim(bottom=1200)
plt.xlim(left=datetime.datetime(1999,6,1))
plt.ylabel('Elo')
plt.title('Top 1v1 ArenaFPS Players\n(other top players in light gray)')
plt.savefig('1v1.png')

In [ ]:
df[(df.t1 == 'ins') | (df.t2 == 'ins')]

In [ ]:
dft = df[df.tier == '1']
for n in (dft.t2.value_counts() + dft.t1.value_counts()).dropna().sort_values(0,False).items():
    print(df[((df.t1 == 'cooller') & (df.t2 == n[0])) | ((df.t1 == n[0]) & (df.t2 == 'cooller'))  ].shape[0],n[0])

In [ ]:
df[((df.t1 == 'cooller') & (df.t2 == 'cypher')) | ((df.t1 == 'cypher') & (df.t2 == 'cooller'))  ]